In [1]:
# For initiating and playing the game
import gym
# for creating dictionary and counter
import collections
# for keeping logs
from tensorboardX import SummaryWriter

In [13]:
# Game environment
# ENV_NAME = "FrozenLake-v0"
ENV_NAME = "FrozenLake8x8-v0"
# Gamma variable used as discount factor
GAMMA = 0.9
# No. of test episodes
TEST_EPISODES = 20

In [10]:
class Agent:
    '''
    Creating a class for agent that will play in the given environment
    '''
    def __init__(self):
        self.env = gym.make(ENV_NAME)
        self.state = self.env.reset()
        self.rewards = collections.defaultdict(float)
        self.transits = collections.defaultdict(
            collections.Counter)
        self.values = collections.defaultdict(float)
    
    def play_n_random_steps(self, count):
        # Playing game with random steps and storing the count of each state
        for _ in range(count):
            action = self.env.action_space.sample()
            new_state, reward, is_done, _ = self.env.step(action)
            self.rewards[(self.state, action, new_state)] = reward
            self.transits[(self.state, action)][new_state] += 1
            self.state = self.env.reset() if is_done else new_state
            
    def calc_action_value(self, state, action):
        # Calculating action value, Q, from the state value, v.
        target_counts = self.transits[(state, action)]
        total = sum(target_counts.values())
        action_value = 0.0
        for tgt_state, count in target_counts.items():
            reward = self.rewards[(state, action, tgt_state)]
            val = reward + GAMMA * self.values[tgt_state]
            action_value += (count/total) * val
        return action_value
    
    def select_action(self, state):
        # Selecting the best action for given state
        best_action, best_value = None, None
        for action in range(self.env.action_space.n):
            action_value = self.calc_action_value(state, action)
            if best_value is None or best_value < action_value:
                best_value = action_value
                best_action = action
        return best_action
    
    def play_episode(self, env):
        # Playing the game with the best actions for each state
        total_reward = 0.0
        state = env.reset()
        while True:
            action = self.select_action(state)
            new_state, reward, is_done, _ = env.step(action)
            self.rewards[(state, action, new_state)] = reward
            self.transits[(state, action)][new_state] += 1
            total_reward += reward
            if is_done:
                break
            state = new_state
        return total_reward
    
    def value_iteration(self):
        # Calculating the action value, Q, and updating the value, V.
        for state in range(self.env.observation_space.n):
            state_values = [
                self.calc_action_value(state, action)
                for action in range(self.env.action_space.n)
            ]
            self.values[state]=max(state_values)

In [14]:
test_env = gym.make(ENV_NAME)
agent = Agent()
writer = SummaryWriter(comment='-8x8v-iteration')

iter_no = 0
best_reward = 0.0
while True:
    iter_no += 1
    # Playing random game 100 times
    agent.play_n_random_steps(100)
    # Updating the values
    agent.value_iteration()
    
    reward = 0.0
    for _ in range(TEST_EPISODES):
        reward += agent.play_episode(test_env)
    # Dividing the reward by no. of total episodes
    reward /= TEST_EPISODES
    writer.add_scalar("reward", reward, iter_no)
    # Printing only the best reward
    if reward > best_reward:
        print("Best reward updated %.3f -> %.3f" % (
            best_reward, reward))
        best_reward = reward
    if reward > 0.80:
        print("Solved in %d iterations!" % iter_no)
        break
writer.close()

Best reward updated 0.000 -> 0.050
Best reward updated 0.050 -> 0.100
Best reward updated 0.100 -> 0.350
Best reward updated 0.350 -> 0.550
Best reward updated 0.550 -> 0.700
Best reward updated 0.700 -> 0.850
Solved in 120 iterations!
